In [9]:
import requests
from bs4 import BeautifulSoup
import WebScrapingMethods as wsm
import pandas as pd


base_url = 'https://www.federalreserve.gov/releases/h41/'


In [10]:
soup = wsm.getSoup(base_url)
hrefs = soup.select('.col-xs-1 a')
end_urls = []
for a in hrefs:
    if (a.attrs['href'][:4] == '2020' or a.attrs['href'][:4] == '2019'
        or a.attrs['href'][:4] == 'curr'):
        end_urls.append(a.attrs['href'] + '/h41.htm')
    else:
        break

In [8]:
urls

['https://www.federalreserve.gov/releases/h41/20200702/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200709/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200716/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200723/h41.htm',
 'https://www.federalreserve.gov/releases/h41/current/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200604/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200611/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200618/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200625/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200507/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200514/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200521/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200528/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200402/h41.htm',
 'https://www.federalreserve.gov/releases/h41/20200409/h41.htm',
 'https://www.federalreser

In [3]:
urls = wsm.getLinks(base_url, end_url_list=end_urls)
dfs = []
for url in urls:
    print(url)
    bs = wsm.getSoup(url)
    data = bs.select('.H41Release > tr > td > p')
    if (len(data) == 0):
        data = bs.select('.H41Release td p')
        if (len(data) == 0):
            print('no data')
            print(url)
            break;
    features = wsm.getFeatures(bs)
    clean_features = wsm.cleanFeatures(features)
    clean_data = wsm.cleanData(data, len(clean_features[2]))
    #print(clean_features)
    #print(clean_data)
    dfs.append(wsm.createDataFrame(clean_features, clean_data))

https://www.federalreserve.gov/releases/h41/20200702/h41.htm
20 36
https://www.federalreserve.gov/releases/h41/20200709/h41.htm
20 36
https://www.federalreserve.gov/releases/h41/20200716/h41.htm
21 36
https://www.federalreserve.gov/releases/h41/20200723/h41.htm
21 36
https://www.federalreserve.gov/releases/h41/current/h41.htm
21 36
https://www.federalreserve.gov/releases/h41/20200604/h41.htm
18 35
https://www.federalreserve.gov/releases/h41/20200611/h41.htm
18 35
https://www.federalreserve.gov/releases/h41/20200618/h41.htm
20 36
https://www.federalreserve.gov/releases/h41/20200625/h41.htm
20 36
https://www.federalreserve.gov/releases/h41/20200507/h41.htm
19 32
https://www.federalreserve.gov/releases/h41/20200514/h41.htm
19 33
https://www.federalreserve.gov/releases/h41/20200521/h41.htm
19 33
https://www.federalreserve.gov/releases/h41/20200528/h41.htm
19 34
https://www.federalreserve.gov/releases/h41/20200402/h41.htm
19 29
https://www.federalreserve.gov/releases/h41/20200409/h41.htm
19

In [7]:
dfs[2]

'None'

In [ ]:
df = dfs[0]
for d in dfs[1:]:
    df = df.append(d, ignore_index=False)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%B %d, %Y', errors='ignore')
df.set_index('Date', inplace=True)

In [ ]:
all_dfs = [df]
columnNames = df.columns
df.fillna('0', inplace=True)

In [ ]:
#clean data frame
for df in all_dfs:
    for c in columnNames:
        df[c] = df[c].apply(removeUnicode)
        df[c] = df[c].apply(removePlus)
        df[c] = df[c].apply(removeComma)
        df[c] = df[c].astype(int)

In [ ]:
namesdict = {}
for i, c in enumerate(df.columns):
    namesdict[c] = c.strip('1234567890').title()
df.rename(namesdict, axis='columns', inplace=True)

In [ ]:
df.to_csv('fedreservesummary_test.csv')